In [1]:
% matplotlib inline
import bidict
import os
import csv
import scipy.sparse
import numpy as np
import random
import matplotlib.pyplot as plt
import load_data
from blessings import Terminal
term = Terminal()
print term.blue('kotek')

kotek


In [4]:
def agnieszka_folds(all_compunds_file, folder_with_pairs):
    possible_compounds = 'possible compounds'
    pairs_contained = 'pairs contained'
    probability = 'probability'
    best_clustering = None
    best_score = 10**9
    scores = []
    varianves = []
    n_ommited_pairs = []
    while(True): # many times
        bin_sim, _, mapping_idx_chembl = load_data.load_similarity_matrices(all_compunds_file, folder_with_pairs)
        bin_sim.setdiag(np.zeros((bin_sim.shape[0]))) # pairs (i,i) are not interesting for us
        pairs = zip(bin_sim.nonzero()[0], bin_sim.nonzero()[1])
                
        random.shuffle(pairs)
        clusters = [{possible_compounds:list(mapping_idx_chembl.keys()), pairs_contained:[]} for i in xrange(2)]
        clusters[0][probability], clusters[1][probability] = (0.0, 0.9), (0.9, 1.0)
        failures = 0
        max_failures = len(pairs)
#         rozwazonych_par = 0
        try:
            while failures < max_failures:
                i, j = pairs.pop() # popping random pair
#                 rozwazonych_par += 1
#                 print term.green + 'rozwazam: (', i, j, ')'+term.normal
                if i > j:
#                     print term.green('odrzucilem')
                    continue # saving time
                
                x = np.random.rand()
#                 print term.blue(str(x))
                for cluster in clusters:  # iterating over clusters to find the chosen one
                    if cluster[probability][0] <= x < cluster[probability][1]:  # if the cluster was chosen
#                         print term.green + 'found cluster with prob:', cluster[probability], term.normal
                        # if pair might go inside
                        if i in cluster[possible_compounds] and j in cluster[possible_compounds]:
#                             print term.green('dorzucilem pare')
                            failures = 0
                            cluster[pairs_contained].append((i,j))
                            
                            # usun zwiazki i, j z mozliwych w kazdym klastrze poza obecnym
                            for c in clusters:
                                if i in c[possible_compounds]:
                                    c[possible_compounds].remove(i)
                                if j in c[possible_compounds]:
                                    c[possible_compounds].remove(j)
                            cluster[possible_compounds].extend([i, j])
                        else: # the cluster was chosen but we cannot fit the pair there
#                             print term.green('nie dorzucilem')
                            failures += 1
                            pairs.insert(0, (i, j)) # pair goes back to the poll
                        break # we've found the chosen one
#                 print term.green + 'failure is:', failures, term.normal
                
        except IndexError:  # zbior par jest pusty
            print term.green('index error')
            pass
        finally:
#             print term.red + 'liczba porazek', failures, term.normal
            # wyszlismy z while'a, wiec albo mamy wpasowane wszystkie pary albo mielismy 100 porazek
            omitted_pairs = len(pairs)
            variance = np.var([float(len(cluster[pairs_contained]))/(cluster[probability][1]-cluster[probability][0])
                               for cluster in clusters])
            score = (5./8)*variance + omitted_pairs # chcemy niską wariancję i mało pominiętych par
            
#             print term.yellow + 'score:', score, '\tvariance:', variance, '\tomitted_pairs:', omitted_pairs, term.normal
            # TODO: new score needed
#             print 'rozwazonych_par', rozwazonych_par
            print 'omitted_pairs', omitted_pairs
            if score < best_score:
                print '\n'
                
                best_score = score
                best_clustering = clusters
                scores.append(best_score)
                varianves.append(variance)
                n_ommited_pairs.append(omitted_pairs)
                plt.plot(range(len(scores)), scores, c='r')
                plt.plot(range(len(varianves)), varianves, c='g')
                plt.plot(range(len(n_ommited_pairs)), n_ommited_pairs, c='b')
                plt.title('best score over time')
                plt.show()
                print best_clustering
                print "score", score, 'variance', variance, 'omitted_pairs', omitted_pairs
                print 'wielkości klastrów', [len(cluster[pairs_contained]) for cluster in clusters]
                print 'pominietych par', omitted_pairs
    
            print '_________________________________________________\n'
    
    return best_clustering

In [ ]:
best_clustering = agnieszka_folds('Random_compounds_100.sdf', 'pairs')

In [ ]:
def staszek_folds():
    pass

In [ ]:
def clustering_score(clusters_sizes, n_omitted):
    variance = some_function(clusters_sizes)
    return 30*varince + n_omitted # TODO 30 - powinno byc wiecej, bo klastrow bedzie mniej

In [ ]:
best_clustering

In [4]:
dir(term)


['__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__format__',
 '__getattr__',
 '__getattribute__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_background_color',
 '_foreground_color',
 '_formatting_string',
 '_height_and_width',
 '_resolve_capability',
 '_resolve_color',
 '_resolve_formatter',
 '_sugar',
 'color',
 'does_styling',
 'fullscreen',
 'height',
 'hidden_cursor',
 'is_a_tty',
 'location',
 'number_of_colors',
 'on_color',
 'width']

In [7]:
from blessings import Terminal as T
t = T()
print dir(t)
print t.blue(str(384))

['__class__', '__delattr__', '__dict__', '__doc__', '__format__', '__getattr__', '__getattribute__', '__hash__', '__init__', '__module__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_background_color', '_does_styling', '_foreground_color', '_formatting_string', '_height_and_width', '_init_descriptor', '_is_a_tty', '_resolve_capability', '_resolve_color', '_resolve_formatter', '_sugar', 'color', 'does_styling', 'fullscreen', 'height', 'hidden_cursor', 'is_a_tty', 'location', 'number_of_colors', 'on_color', 'stream', 'width']
384
